In [ ]:
# 调用Deepseek-R1示例，支持流式输出和思考链
from openai import OpenAI

client = OpenAI(
	base_url="https://ai.gitee.com/v1",
	api_key="DUQFR61KA8QLDVEQPGJKBXYSL2DXMPST1FM98Y1L",
	default_headers={"X-Failover-Enabled":"true"},
)

response = client.chat.completions.create(
	messages=[
		{
			"role": "system",
			"content": "You are a helpful and harmless assistant. You should think step-by-step."
		},
		{
			"role": "user",
			"content": "Can you please let us know more details about your "
		}
	],
	model="DeepSeek-R1",
	stream=True,
	max_tokens=8192,
	temperature=0.6,
	top_p=0.7,
	extra_body={
		"top_k": 50,
	},
	frequency_penalty=0,
)

fullResponse = ""
print("Response:")
# Print streaming response
for chunk in response:
	if len(chunk.choices) == 0:
		continue
	delta = chunk.choices[0].delta
	# If is thinking content, print it in gray
	if hasattr(delta, 'reasoning_content') and delta.reasoning_content:
		fullResponse += delta.reasoning_content
		print(f"\033[90m{delta.reasoning_content}\033[0m", end="", flush=True)
	elif delta.content:
		fullResponse += delta.content
		print(delta.content, end="", flush=True)

# 1、调用API

In [4]:
# --- API ---
from openai import OpenAI

client = OpenAI(
    base_url="https://ai.gitee.com/v1",
    api_key="DUQFR61KA8QLDVEQPGJKBXYSL2DXMPST1FM98Y1L",
    default_headers={"X-Failover-Enabled":"true"},
)

# 2、加载处理的prompt.txt

In [5]:
# Load prompt from text file
def load_text(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

SYSTEM_PROMPT = load_text("law_prompt.txt")  # 这里用你的prompt .txt

# 3、抽取知识的关系并输出json文件

In [ ]:
# 抽取知识图谱
import re
import json
# 从一大串字符串中“抠出”最外层 JSON
def extract_json_from_string(s: str) -> str:
    """
    在字符串中找到第一个 { 到 最后一个 } 之间的内容，
    认为这是 JSON 对象，返回这段子串。
    """
    s = s.strip()
    # 用正则搜索最外层的大括号
    m = re.search(r'\{.*\}', s, re.S)
    if not m:
        raise ValueError("在模型输出中没有找到 JSON 对象，原始内容为：\n" + s)
    return m.group(0)

# 4. 用流式的方式调用 DeepSeek-R1，抽取知识图谱
def extract_kg_stream(text: str):
    response = client.chat.completions.create(
        model="DeepSeek-R1",
        stream=True,                      # ✅ 保留流式
        response_format={"type": "json_object"},  # ✅ 告诉服务端我们要 JSON
        max_tokens=1500,
        temperature=0.2,
        top_p=0.7,
        extra_body={
            "top_k": 50,
            "enable_reasoning": False,     # ✅ 关闭长时间推理，避免一直跑
        },
        messages=[
            {
                "role": "system",
                "content": SYSTEM_PROMPT
            },
            {
                "role": "user",
                "content": f"请从以下文本中抽取知识图谱实体和关系：\n\n{text}"
            }
        ],
    )

    full_json_str = ""
    print("Response:")

    # 5. 流式逐块接收
    for chunk in response:
        if len(chunk.choices) == 0:
            continue

        delta = chunk.choices[0].delta

        # 如果有推理内容（reasoning_content），只打印，不参与 JSON
        if hasattr(delta, "reasoning_content") and delta.reasoning_content:
            # 灰色输出思考过程，仅供你调试看
            print(f"\033[90m{delta.reasoning_content}\033[0m", end="", flush=True)

        # 真正的 JSON 字符串在 content 里
        elif delta.content:
            full_json_str += delta.content
            # 你也可以边打印边看模型在输出什么
            print(delta.content, end="", flush=True)

    print("\n\n=== Raw streamed content ===")
    print(full_json_str)

    # 6. 从整串内容里抠出 JSON 部分
    json_str = extract_json_from_string(full_json_str)
    print("\n=== Extracted JSON string ===")
    print(json_str)

    # 7. 解析为 Python dict
    kg_data = json.loads(json_str)
    print("\n✅ JSON 解析成功！")
    return kg_data

In [ ]:
# 测试文本
test_text = """
第十条 著作权包括下列权利：
（一）发表权，即决定作品是否公之于众的权利；
（二）署名权，即表明作者身份，在作品上署名的权利；
（三）修改权，即修改或者授权他人修改作品的权利；
（四）保护作品完整权。
"""

kg = extract_kg_stream(test_text)

print("\n=== 最终解析结果（美化输出） ===")
print(json.dumps(kg, ensure_ascii=False, indent=2))

Response:
首先，用户提供了一个文本："第十条 著作权包括下列权利：（一）发表权，即决定作品是否公之于众的权利；（二）署名权，即表明作者身份，在作品上署名的权利；（三）修改权，即修改或者授权他人修改作品的权利；（四）保护作品完整权。"

我的任务是提取知识图谱的实体和关系，并输出为JSON格式。JSON必须有"entities"和"relations"数组。entities中的每个对象有"uid"、"node_type"、"name"、"extra"。relations中的每个对象有"from_uid"、"to_uid"、"rel_type"。

规则：
- uid必须唯一且稳定。我应该基于内容创建uid。
- node_type可以是["Law","Chapter","Article","Right","Concept","Definition","Case","Entity"]。
- extra是可选键值对。
- rel_type必须大写且简洁。
- 输出时，只在"content"字段中放JSON，不要有其他解释。步骤推理放在"reasoning_content"中。
- 输出必须是纯JSON，没有markdown前缀如```json。

用户说："输出格式要求：1. 以json格式输出，注意不要带有```json这样的markdown格式前缀"。所以，我必须直接输出JSON对象。

现在，分析文本：
- 文本是"第十条"，这看起来是一个法律条文的一部分，具体是著作权法中的第10条。
- 它定义了著作权包括的权利：发表权、署名权、修改权、保护作品完整权。
- 每个权利后有解释。

我需要提取实体和关系。

可能的实体：
1. 法律条文本身："第十条"。node_type应该是"Article"，因为它是法律的条款。
   - uid: "Article:第10条"（类似例子中的"Article:第10条"）
   - name: "第十条"
   - extra: 可能包括文本内容？extra是可选的，我可以放一些信息，但不要冗余。

2. 这些权利：发表权、署名权、修改权、保护作品完整权。
   - node_type应为"Right"，因为它们是在定义权利。
   - uid: 唯一标识符，例如"Right:发表权"、"Right:署名权"等。
  

# 4、保存生成的JSON文件

In [8]:
import os
import json

# 你想保存 JSON 的目录（改成你自己的）
OUTPUT_DIR = "D:\\Desktop\\现代软件工程\\作业\\实践营\\KG\\KG_files\\KG_json_test"
# 如果目录不存在就创建
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
def save_kg_json(kg: dict, output_dir: str, filename: str):
    """把 kg（Python 字典）保存为一个 .json 文件"""
    path = os.path.join(output_dir, filename)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(kg, f, ensure_ascii=False, indent=2)
    print(f"已保存到: {path}")

In [10]:
# 从 kg 里取出条文 uid 或 name，当作文件名一部分
article_uid = None
for ent in kg["content"]["entities"]:
    if ent["node_type"] == "Article":
        article_uid = ent["uid"]  # 比如 "Article:第10条"
        break

if article_uid is None:
    filename = "unknown_article.json"
else:
    # 简单处理下文件名里的冒号
    safe_uid = article_uid.replace(":", "_")
    filename = f"{safe_uid}.json"   # Article_第10条.json

save_kg_json(kg, OUTPUT_DIR, filename)


已保存到: D:\Desktop\现代软件工程\作业\实践营\KG\KG_files\KG_json_test\Article_第10条.json
